# Push data to ES

## Major

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
import pandas as pd
import os

PATH = '../data/test_major'
docs = pd.read_csv(f"{PATH}/docs.csv")
docs = docs[docs['id'] >= 6]
conf = Configuration()
# docs = docs[:20]
docs.shape

(39, 4)

In [2]:
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter

db_df = docs
db = []
dist = []

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n","\n\n", "\n"],
    chunk_size=1800,
    chunk_overlap=120,
    length_function=len,
    is_separator_regex=False,
)

for i, data in db_df.iterrows():
    doc = text_splitter.create_documents([data['content']])
    dist.append(len(doc))
    [d.metadata.update({"id": data['id']}) for d in doc]
    db.extend(doc)

print(len(db))
Counter(dist)

92


Counter({2: 24, 3: 13, 1: 1, 4: 1})

### Gemini

In [3]:
INDEX = "gemini-major"

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = conf.load_gemini_token()
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# a = embeddings.embed_documents(["AI", "CNTT", "Arts"])
# a

### VN SBert

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "vnsbert-major"

MODEL = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(0),
    model_name=MODEL)

### LaBSE

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "test-db"

MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(1),
    model_name=MODEL)

## Load

In [6]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=conf.load_elasticsearch_connection(),
                          index_name=INDEX,
                          distance_strategy="EUCLIDEAN_DISTANCE")